## Databases for each n-gram combination

There are 22 different n-gram combinations (1-3 grams) with maximum context 6. This notebook constructs a database for each type of n-gram and also a database storing the counts for each n-gram type.

In [14]:
%matplotlib inline

In [15]:
import itertools
import pickle
from collections import defaultdict
import numpy as np
import pickle
import pandas as pd
import os
import time
import dill
from glob import iglob
import multiprocessing

In [16]:
context = 6
combinations = []
for n_gram in range(1, 4):
    combinations += [[0] + list(tup) for tup in itertools.combinations(range(1, context), n_gram-1)]

In [4]:
combinations = ["".join(str(num) for num in combination) for combination in combinations]
len(combinations)

16

In [5]:
def getTotalBscore(bscore_file):
    bscore_array = []
    with open(bscore_file,'rb') as f:
        bscore_array = pickle.load(f)
    total_bscore = np.array([]).reshape(62,0)
    for page in bscore_array:
        try:
            total_page = unpackbits(np.array(page), 62)
        except TypeError:
            total_page = np.array([]).reshape(62,0)
            for num in page:
                col = np.array(decodeColumn(num)).reshape(62,-1)
                total_page = np.concatenate((total_page,col),axis=1)
        total_bscore = np.concatenate((total_bscore,total_page),axis=1)
    return total_bscore

def decodeColumn(num):
    col = []
    for i in range(62):
        col.insert(0,num%2)
        num = int(num/2)
    return col

def unpackbits(x, num_bits):
    xshape = list(x.shape)
    x = x.reshape([-1, 1])
    mask = 2**np.arange(num_bits, dtype=x.dtype).reshape([1, num_bits])
    return np.flip((x & mask).astype(bool).astype(float), 1).T

In [6]:
def decodeColumn(num):
    col = []
    for i in range(62):
        col.insert(0,num%2)
        num = int(num/2)
    return col

In [7]:
def unpackbits(x, num_bits):
    xshape = list(x.shape)
    x = x.reshape([-1, 1])
    mask = 2**np.arange(num_bits, dtype=x.dtype).reshape([1, num_bits])
    return np.flip((x & mask).astype(bool).astype(float), 1).T

We precompute powers of 2 from $2^0$ to $2^{61}$ to speed up calculating hash values.

In [8]:
powers = 1 << np.arange(62)[::-1]

In [9]:
def make_fps(data, combinations, dbs, piece):
    for colindex in range(len(data)):
        for combination in combinations:
            cols = []
            # we need at least enough fingerprints for all the indices in our combination
            try:
                for i in combination:
                    cols.append(data[colindex+int(i)])
            except IndexError:
                continue
            fp = []
            equals_Zero = True
            for column in cols:
                hashint = int(column.dot(powers))
                fp.append(hashint)
                if hashint != 0:
                    equals_Zero = False
            if equals_Zero == True:
                continue
            dbs[combination][tuple(fp)][piece].append(colindex)

Now we iterate over every query in our list of queries, compute the bootleg score for that query, generate all possible n-grams from the bootleg score, and store it in our database, which is an in-memory Python dictionary.

In [10]:
def make_DB(filelist, combinations, idx = 1, dbs = None):
    if not dbs:
        dbs = {combination: defaultdict(lambda : defaultdict(list)) for combination in combinations}
    with open(filelist, 'r') as f:
        failed = []
        for i, curfile in enumerate(f):
            print(f'{idx}-{i}')
            curfile = curfile.strip().strip('\n')
            try:
                num = curfile.split('/')[-1][0]
                if num == 'd':
                    data = getTotalBscore(curfile)
                else:
                    with open(curfile, 'rb') as pickle_file:
                        data = pickle.load(pickle_file)
                make_fps(data.T, combinations, dbs, i)
            except:
                failed.append(curfile)
    return dbs

In [11]:
def store_DB(db, combination, outdir):
    with open(f"{outdir}/{combination}.pkl", "wb") as f:
        dill.dump(db, f)

In [12]:
filelist = 'cfg_files/db.list'

In [20]:
db_dir = '/data1/dyang/MarketplaceRepeat'

In [21]:
os.makedirs(db_dir, exist_ok=True)

We make a mapping from a number to each piece to save memory.

In [22]:
# num_to_piece = {}

In [23]:
# with open(filelist, 'r') as f:
#     failed = []
#     for i, curfile in enumerate(f):
#         curfile = curfile.strip().strip('\n')
#         piece = curfile.split('/')[-1][:-4]
#         num_to_piece[i] = piece

In [24]:
# with open("num_to_piece_random.pkl", "wb") as f:
#     pickle.dump(num_to_piece, f)

In [25]:
with open("num_to_piece_random.pkl", 'rb') as f:
    num_to_piece = pickle.load(f)

Here we create a database for each n-gram type (total 16). Each n-gram database maps from the n-gram (a tuple) to a dictionary. This dictionary maps each piece that the n-gram appears in to a list of offsets within the piece.

In [26]:
# for i in range(0, len(combinations), 3):
#     print(f'Combinations {i}-{i+3}')
#     dbs = make_DB(filelist, combinations[i: i+3])
#     for combination in dbs:
#         store_DB(dbs[combination], combination, db_dir)
#     dbs.clear()

In [27]:
def parallelize(i):
    print(f'Processing {i}')
    dbs = make_DB(filelist, combinations[i:i+1], idx = i)
    for combination in dbs:
        store_DB(dbs[combination], combination, db_dir)
    dbs.clear()

In [ ]:
inputs = [i for i in range(len(combinations))]
ncores = 3
pool = multiprocessing.Pool(ncores)
outputs = list(pool.map(parallelize, inputs))

Processing 0
Processing 2
Processing 4
0-0
2-0
4-0
4-1
0-1
2-1
0-2
4-2
2-2
0-3
2-3
4-3
0-4
2-4
4-4
0-5
0-6
0-7
0-8
4-5
2-5
4-6
2-6
4-7
2-7
4-8
2-8
0-9
0-10
0-11
0-12
4-9
4-10
4-11
4-12
2-9
2-10
2-11
2-12
0-13
0-14
0-15
0-16
0-17
0-18
4-13
4-14
4-15
2-13
0-19
2-14
2-15
4-16
4-17
4-18
2-16
2-17
2-18
4-19
2-19
0-20
0-21
4-20
4-21
2-20
2-21
0-22
4-22
0-23
2-22
0-24
4-23
4-24
0-25
4-25
2-23
2-24
2-25
0-26
0-27
4-26
4-27
0-28
4-28
2-26
2-27
0-29
0-30
2-28
0-31
4-29
4-30
0-32
4-31
4-32
0-33
2-29
2-30
4-33
2-31
2-32
2-33
4-34
4-35
4-36
4-37
2-34
2-35
2-36
2-37
0-34
0-35
0-36
0-37
0-38
0-39
4-38
0-40
0-41
4-39
4-40
4-41
0-42
0-43
0-44
0-45
2-38
2-39
4-42
4-43
4-44
2-40
2-41
4-45
0-46
0-47
2-42
0-48
2-43
2-44
2-45
4-46
4-47
4-48
0-49
2-46
2-47
0-50
2-48
4-49
4-50
2-49
0-51
2-50
0-52
0-53
0-54
0-55
0-56
0-57
0-58
4-51
0-59
4-52
4-53
4-54
4-55
4-56
0-60
4-57
4-58
0-61
0-62
2-51
0-63
0-64
4-59
2-52
2-53
2-54
2-55
2-56
0-65
2-57
4-60
2-58
4-61
4-62
2-59
4-63
4-64
2-60
4-65
2-61
2-62
2-63
2-64
0-66
0

2-455
0-498
0-499
4-461
0-500
4-462
2-456
0-501
4-463
0-502
0-503
0-504
4-464
0-505
4-465
4-466
4-467
4-468
4-469
4-470
4-471
4-472
4-473
0-506
2-457
2-458
0-507
0-508
2-459
0-509
0-510
4-474
0-511
0-512
4-475
2-460
4-476
4-477
4-478
4-479
4-480
2-461
2-462
4-481
2-463
4-482
4-483
4-484
2-464
2-465
0-513
0-514
2-466
4-485
0-515
4-486
4-487
0-516
2-467
2-468
0-517
0-518
0-519
2-469
0-520
0-521
4-488
0-522
2-470
2-471
2-472
2-473
0-523
0-524
0-525
2-474
2-475
2-476
2-477
2-478
4-489
2-479
4-490
0-526
4-491
0-527
4-492
0-528
2-480
0-529
0-530
4-493
4-494
0-531
0-532
2-481
4-495
2-482
2-483
2-484
0-533
4-496
4-497
2-485
2-486
2-487
2-488
0-534
0-535
0-536
0-537
2-489
0-538
0-539
2-490
4-498
2-491
2-492
0-540
4-499
2-493
0-541
4-500
2-494
0-542
4-501
2-495
4-502
0-543
4-503
4-504
0-544
0-545
0-546
2-496
2-497
4-505
0-547
0-548
0-549
0-550
4-506
0-551
4-507
4-508
0-552
4-509
0-553
4-510
0-554
0-555
4-511
4-512
2-498
2-499
2-500
2-501
2-502
4-513
4-514
0-556
0-557
4-515
2-503
4-516
2-504
4-51

0-973
0-974
4-910
2-897
4-911
0-975
0-976
0-977
0-978
0-979
4-912
4-913
4-914
4-915
4-916
0-980
0-981
0-982
4-917
4-918
4-919
4-920
4-921
4-922
4-923
2-898
4-924
4-925
2-899
2-900
4-926
2-901
0-983
2-902
4-927
0-984
4-928
4-929
2-903
2-904
0-985
0-986
0-987
4-930
0-988
2-905
0-989
4-931
4-932
2-906
2-907
2-908
2-909
0-990
0-991
0-992
2-910
2-911
4-933
0-993
0-994
0-995
0-996
0-997
0-998
4-934
0-999
2-912
0-1000
2-913
2-914
2-915
4-935
2-916
0-1001
4-936
4-937
0-1002
4-938
4-939
4-940
2-917
2-918
4-941
2-919
2-920
4-942
2-921
2-922
2-923
4-943
4-944
2-924
2-925
4-945
2-926
4-946
2-927
4-947
2-928
2-929
0-1003
0-1004
2-930
0-1005
2-931
2-932
0-1006
0-1007
4-948
4-949
4-950
0-1008
0-1009
4-951
2-933
2-934
4-952
4-953
0-1010
0-1011
0-1012
4-954
4-955
4-956
4-957
4-958
2-935
4-959
0-1013
4-960
4-961
4-962
0-1014
2-936
2-937
0-1015
2-938
2-939
4-963
4-964
2-940
4-965
0-1016
4-966
0-1017
4-967
2-941
2-942
4-968
4-969
2-943
2-944
0-1018
4-970
0-1019
2-945
2-946
4-971
2-947
4-972
0-1020
0-1021


## Make fingerprint matches table

The fingerprint matches table is a nested dictionary. 
* The first level maps each n-gram type to another dictionary. 
* The second dictionary maps each n-gram of that type to a tuple (x, y), where x is the total number of times the fingerprint occurs in IMSLP and y is the total number of PDFs in IMSLP with that fingerprint.

In [5]:
db_dir = '/data1/dyang/MarketplaceConvert'

In [9]:
fp_matches = {combination: {} for combination in combinations}

In [ ]:
for combination in combinations:
    with open(f"{db_dir}/{combination}.pkl", "rb") as f:
        d = dill.load(f)
    print(f"{combination} {len(d)}")
    for fp in d:
        # first element is number of times the fingerprint occurs, second element is number of PDFs containing the fingerprint
        fp_matches[combination][fp] = (sum(len(d[fp][piece]) for piece in d[fp]), len(d[fp]))
    d.clear()

0 1
01 1


In [18]:
list(d.keys())

[(1100585369600,)]

In [8]:
fp_matches_file = f"{db_dir}/fp_matches.pkl"
with open(fp_matches_file, "wb") as f:
    pickle.dump(fp_matches, f)

In [19]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
